# Module 02: Platform Setup (Upwork & Payment Methods)

**Difficulty**: ⭐

**Estimated Time**: 60 minutes

**Prerequisites**: [Module 01: Portfolio Development](01_portfolio_development.ipynb)

## Learning Objectives

By the end of this notebook, you will be able to:
1. Create an optimized Upwork profile that attracts clients
2. Set up Wise payment account for maximum fee savings
3. Understand platform fees and pricing strategy
4. Write a compelling profile bio
5. Position yourself in the right niche

## Setup

In [ ]:
from pathlib import Path
import json

# Platform fee structures
PLATFORM_FEES = {
    'upwork': 0.10,  # 10% flat rate
    'fiverr': 0.20,  # 20% commission
    'freelancer': 0.10,  # 10%
}

PAYMENT_FEES = {
    'wise': {'conversion': 0.005, 'transfer': 1.06},  # 0.5% + RM1.06
    'payoneer': {'conversion': 0.005, 'transfer': 0},  # ~0.5%
    'paypal': {'transaction': 0.044, 'conversion': 0.04, 'withdrawal': 50},  # Expensive!
}

USD_TO_MYR = 4.3

print("✓ Platform setup tools loaded")

## Part 1: Understanding Platform Economics

### Calculate True Take-Home Pay

In [ ]:
def calculate_take_home(gross_usd, platform='upwork', payment_method='wise'):
    """
    Calculate actual money you receive after all fees.
    
    Args:
        gross_usd: Amount client pays
        platform: 'upwork', 'fiverr', or 'freelancer'
        payment_method: 'wise', 'payoneer', or 'paypal'
    
    Returns:
        Breakdown of fees and final amount in RM
    """
    # Platform fee
    platform_fee = gross_usd * PLATFORM_FEES[platform]
    after_platform = gross_usd - platform_fee
    
    # Convert to MYR
    rm_amount = after_platform * USD_TO_MYR
    
    # Payment processing fees
    payment_config = PAYMENT_FEES[payment_method]
    
    if payment_method == 'paypal':
        transaction_fee = after_platform * payment_config['transaction']
        conversion_fee = after_platform * payment_config['conversion']
        withdrawal_fee = payment_config['withdrawal']
        total_payment_fees_rm = (transaction_fee + conversion_fee) * USD_TO_MYR + withdrawal_fee
    else:
        conversion_fee_rm = rm_amount * payment_config['conversion']
        transfer_fee_rm = payment_config.get('transfer', 0)
        total_payment_fees_rm = conversion_fee_rm + transfer_fee_rm
    
    final_rm = rm_amount - total_payment_fees_rm
    
    return {
        'gross_usd': gross_usd,
        'platform_fee_usd': platform_fee,
        'after_platform_usd': after_platform,
        'rm_before_payment_fees': rm_amount,
        'payment_fees_rm': total_payment_fees_rm,
        'final_rm': final_rm,
        'total_fees_rm': (gross_usd * USD_TO_MYR) - final_rm,
        'retention_pct': (final_rm / (gross_usd * USD_TO_MYR)) * 100,
    }

# Example: Compare payment methods for RM690 target
target_rm = 690
target_usd = target_rm / USD_TO_MYR

# Need to earn more than target to account for fees
gross_needed_usd = 185  # Approximately to net RM690

print("PAYMENT METHOD COMPARISON")
print(f"Target: RM{target_rm} net")
print(f"Gross needed: ~${gross_needed_usd} USD")
print("="*70)

for payment_method in ['wise', 'payoneer', 'paypal']:
    result = calculate_take_home(gross_needed_usd, 'upwork', payment_method)
    
    print(f"\n{payment_method.upper()}:")
    print(f"  Gross: ${result['gross_usd']} USD")
    print(f"  Platform fee (10%): ${result['platform_fee_usd']:.2f}")
    print(f"  Payment fees: RM{result['payment_fees_rm']:.2f}")
    print(f"  Final take-home: RM{result['final_rm']:.2f}")
    print(f"  You retain: {result['retention_pct']:.1f}% of gross")
    
    if result['final_rm'] >= target_rm:
        print(f"  ✓ Exceeds target by RM{result['final_rm'] - target_rm:.2f}")
    else:
        print(f"  ✗ Short by RM{target_rm - result['final_rm']:.2f}")

print("\n" + "="*70)
print("\nRECOMMENDATION: Use Wise or Payoneer")
print("Avoid PayPal - saves you RM100+ per payment!")

### Exercise 1: Calculate Your Pricing

**Task**: Figure out what hourly rate you need to charge to hit your target after fees.

In [ ]:
def calculate_required_rate(target_rm_monthly, billable_hours_monthly, 
                           platform='upwork', payment_method='wise'):
    """
    Calculate hourly rate needed to hit target income.
    Works backwards from desired take-home.
    """
    # Account for payment fees first (approximate)
    payment_factor = 0.995 if payment_method in ['wise', 'payoneer'] else 0.85
    
    # Account for platform fees
    platform_factor = 1 - PLATFORM_FEES[platform]
    
    # Work backwards
    rm_needed_before_payment_fees = target_rm_monthly / payment_factor
    usd_needed_after_platform = rm_needed_before_payment_fees / USD_TO_MYR
    usd_gross_needed = usd_needed_after_platform / platform_factor
    
    hourly_rate = usd_gross_needed / billable_hours_monthly
    
    return {
        'hourly_rate_usd': hourly_rate,
        'hourly_rate_rm': hourly_rate * USD_TO_MYR,
        'monthly_gross_usd': usd_gross_needed,
        'expected_net_rm': target_rm_monthly,
    }

# YOUR calculation - customize these!
my_target = 690  # RM
my_expected_hours = 15  # billable hours per month

scenarios = [
    {'hours': 12, 'label': 'Conservative (12 hrs/month)'},
    {'hours': 15, 'label': 'Realistic (15 hrs/month)'},
    {'hours': 18, 'label': 'Optimistic (18 hrs/month)'},
]

print(f"REQUIRED HOURLY RATES FOR RM{my_target}/MONTH")
print("="*70)

for scenario in scenarios:
    result = calculate_required_rate(my_target, scenario['hours'])
    
    print(f"\n{scenario['label']}:")
    print(f"  Required rate: ${result['hourly_rate_usd']:.2f}/hr USD (RM{result['hourly_rate_rm']:.2f}/hr)")
    print(f"  Monthly gross: ${result['monthly_gross_usd']:.2f} USD")
    print(f"  Expected net: RM{result['expected_net_rm']:.2f}")
    
    # Assess feasibility
    if result['hourly_rate_usd'] <= 12:
        print(f"  ✓✓ Very achievable at beginner rates")
    elif result['hourly_rate_usd'] <= 18:
        print(f"  ✓ Achievable within 2-3 months")
    else:
        print(f"  → May need to increase billable hours")

print("\n" + "="*70)
print("\nSTRATEGY: Start at $10-12/hr, focus on getting 15+ billable hours")

## Part 2: Creating Your Upwork Profile

### The Profile Formula

In [ ]:
# Profile title templates
title_templates = [
    "Python Data Analyst | Django API Development & Automation Specialist",
    "Data Science Student | Python, Web Scraping & Data Visualization Expert",
    "Django Developer | REST APIs, Data Analysis & Python Automation",
    "Python Specialist | Data Analysis, Web Scraping & Excel Automation",
]

# Profile bio template
bio_template = '''I help businesses automate workflows and extract insights from data using Python.

**What I Do:**
• Data analysis and visualization with Pandas, Matplotlib, and Plotly
• Django REST API development with authentication and testing
• Web scraping and data collection (Beautiful Soup, Scrapy)
• Python automation scripts for Excel, reports, and workflows

**My Approach:**
I deliver clean, well-documented code with clear explanations. Every project includes:
- Comprehensive README with setup instructions
- Error handling and edge case management
- Regular progress updates
- Fast turnaround (typically 1-3 days for small projects)

**Background:**
Data Science student with expertise in Python, Django, and data analysis. I've completed projects in customer segmentation, sales forecasting, and automated reporting systems.

**Let's work together!** I'm available for:
→ Fixed-price projects ($50-300)
→ Hourly consulting for technical problem-solving
→ Ongoing retainer arrangements

Available weekends and evenings (UTC+8 timezone). Usually respond within 2-4 hours.
'''

print("UPWORK PROFILE TEMPLATES")
print("="*70)
print("\nPROFILE TITLE OPTIONS:")
for i, title in enumerate(title_templates, 1):
    print(f"{i}. {title}")

print("\n" + "="*70)
print("\nPROFILE BIO TEMPLATE:")
print("-"*70)
print(bio_template)

print("="*70)
print("\nCUSTOMIZATION TIPS:")
print("• Use specific skills, not generic titles")
print("• Lead with what you DO, not who you ARE")
print("• Include availability and timezone")
print("• Keep under 300 words (clients skim quickly)")
print("• Mention specific project types you handle")

### Exercise 2: Write Your Profile Bio

**Task**: Customize the template above with YOUR skills and background.

In [ ]:
# YOUR profile details - customize this!
my_profile = {
    'title': 'Python Data Analyst | Django & Automation Specialist',  # Choose from templates above
    
    'core_skills': [
        'Data analysis with Pandas and NumPy',
        'Django REST API development',
        'Web scraping (Beautiful Soup)',
        'Python automation scripts',
        'Data visualization (Matplotlib, Plotly)',
    ],
    
    'background': 'Data Science student with experience in Python development and statistical analysis',
    
    'unique_value': 'Fast turnaround with well-documented, production-ready code',
    
    'availability': 'Weekends and evenings (UTC+8 timezone)',
    
    'project_types': [
        'Data cleaning and preprocessing',
        'Web scraping projects',
        'Django bug fixes and API endpoints',
        'Excel automation with Python',
    ],
}

def generate_professional_bio(profile_dict):
    """Generate a complete Upwork bio from profile dictionary."""
    bio = f"""I help businesses extract insights from data and automate workflows using Python.

**Core Skills:**
"""
    for skill in profile_dict['core_skills']:
        bio += f"• {skill}\n"
    
    bio += f"\n**What Makes Me Different:**\n{profile_dict['unique_value']}\n\n"
    bio += f"**Background:**\n{profile_dict['background']}\n\n"
    bio += f"**Project Types I Excel At:**\n"
    
    for ptype in profile_dict['project_types']:
        bio += f"→ {ptype}\n"
    
    bio += f"\n**Availability:** {profile_dict['availability']}\n"
    bio += "\nI typically respond within 2-4 hours and deliver small projects in 1-3 days."
    
    return bio

# Generate your bio
my_bio = generate_professional_bio(my_profile)

print("YOUR UPWORK PROFILE BIO")
print("="*70)
print(f"\nTitle: {my_profile['title']}")
print("\n" + "-"*70)
print(my_bio)
print("="*70)

# Save for later use
profile_file = Path('../templates/upwork_profile.txt')
profile_file.parent.mkdir(exist_ok=True)
with open(profile_file, 'w') as f:
    f.write(f"TITLE:\n{my_profile['title']}\n\n")
    f.write(f"BIO:\n{my_bio}")

print(f"\n✓ Profile saved to: {profile_file}")
print("\nCopy this to your Upwork profile during setup!")

## Part 3: Setting Up Payment Methods

### Step-by-Step: Wise Account Setup

In [ ]:
wise_setup_steps = {
    'Step 1': {
        'action': 'Create Wise account',
        'url': 'https://wise.com/my',
        'time': '5 minutes',
        'details': [
            'Click "Sign up" on Wise Malaysia website',
            'Enter email and create password',
            'Verify email address',
        ],
    },
    'Step 2': {
        'action': 'Complete identity verification',
        'time': '10 minutes',
        'details': [
            'Provide MyKad (Malaysian ID) information',
            'Upload clear photo of MyKad',
            'Take selfie for verification',
            'Verification usually completes within 1-2 days',
        ],
    },
    'Step 3': {
        'action': 'Add Malaysian bank account',
        'time': '5 minutes',
        'details': [
            'Go to "Recipients" → "Add Recipient"',
            'Select Malaysia',
            'Enter your bank details (Maybank, CIMB, etc.)',
            'This is where Wise will transfer your money',
        ],
    },
    'Step 4': {
        'action': 'Get USD receiving details',
        'time': '2 minutes',
        'details': [
            'Click "Account" → "USD" → "Add"',
            'Wise generates US bank details for you',
            'Copy routing number and account number',
            'Give these to Upwork clients for payment',
        ],
    },
}

print("WISE ACCOUNT SETUP GUIDE")
print("="*70)

total_time = 0
for step_name, details in wise_setup_steps.items():
    print(f"\n{step_name}: {details['action']}")
    print(f"Time needed: {details['time']}")
    if 'url' in details:
        print(f"URL: {details['url']}")
    print("Tasks:")
    for task in details['details']:
        print(f"  ☐ {task}")
    
    # Extract time (simple approach)
    time_mins = int(details['time'].split()[0])
    total_time += time_mins

print(f"\n{'='*70}")
print(f"Total setup time: ~{total_time} minutes")
print("\nIMPORTANT: You don't need to deposit money! Wise is free to set up.")
print("You'll only use it to RECEIVE payments from clients.")

## Part 4: Upwork Profile Checklist

### Complete Profile Setup

In [ ]:
upwork_checklist = {
    'Profile Basics': [
        'Professional photo (headshot, smiling, good lighting)',
        'Compelling title (from templates above)',
        'Bio under 300 words (saved in templates folder)',
        'Hourly rate: $10-12/hr to start',
    ],
    'Skills Section': [
        'Add 10-15 relevant skills (Python, Pandas, Django, etc.)',
        'Focus on in-demand skills that match your projects',
        'Upwork will suggest skills - pick relevant ones',
    ],
    'Portfolio': [
        'Add 3-5 GitHub projects with descriptions',
        'Use project titles from Module 01',
        'Include link to live demos if available',
        'Write 2-3 sentence description for each',
    ],
    'Employment History': [
        'Can add relevant coursework as "experience"',
        'Frame as: "Data Analysis Projects" or "Python Development"',
        'Optional: Add teaching assistant or internship roles',
    ],
    'Education': [
        'Add your university and degree program',
        'Include expected graduation year',
        'Mention relevant coursework (if space allows)',
    ],
    'Profile Settings': [
        'Set availability: "As Needed - Open to Offers"',
        'Timezone: UTC+8 (Malaysia)',
        'Visibility: "Public" (so clients can find you)',
        'Enable email notifications for job matches',
    ],
}

print("UPWORK PROFILE COMPLETION CHECKLIST")
print("="*70)

for section, items in upwork_checklist.items():
    print(f"\n{section.upper()}:")
    for item in items:
        print(f"  ☐ {item}")

print(f"\n{'='*70}")
print("\nPROFILE STRENGTH TARGET: 80-100%")
print("Upwork shows profile strength meter - aim for 80%+ before applying")
print("\nGOAL: Complete your profile in ONE sitting (60-90 minutes)")

## Summary

### What You've Learned

1. ✓ Platform fees and payment methods comparison
2. ✓ How to calculate required hourly rates
3. ✓ Upwork profile optimization strategy
4. ✓ Professional bio writing
5. ✓ Wise payment setup process

### Key Takeaways

- **Use Wise or Payoneer** - saves RM100+ per payment vs PayPal
- **Start at $10-12/hr** - competitive for beginners, good income in RM
- **Profile title matters** - be specific, not generic
- **80%+ profile strength** - complete all sections before applying
- **Setup takes 60-90 minutes** - do it in one session

### Your Action Items This Week

1. ☐ Create Wise account and complete verification
2. ☐ Set up Upwork profile with customized bio
3. ☐ Add 3-5 portfolio projects to Upwork
4. ☐ Reach 80%+ profile completion
5. ☐ Save profile bio to templates folder
6. ☐ Set hourly rate at $10-12/hr

### What's Next?

**Module 03: Winning Proposals** - Learn how to write proposals that get responses and land your first projects

### Additional Resources

- Upwork Profile Tips: Search "how to optimize upwork profile" on YouTube
- Wise Help Center: https://wise.com/help/
- Upwork Success Stories: Read profiles of Top Rated freelancers in your niche